In [1]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [2]:
api_key = input("Please enter your YouTube Data API key: ")
channel_ids=['UCX6OQ3DkcsbYNE6H8uQQuVA',
             'UCChmJrVa8kDg05JfCmxpLRw',
             'UCkX0NXCP9kCLtk30kjTXIWg',
             'UCXqJrBG563pZnaqLoxHjUDg',
             'UCHoLIMtg_OigNlJmjHX9J8Q',
             'UCMOLiZpKXp-w5CNfqJObQUw',
             'UCtAXJ4DNpshfVjLIMq9pIRw']

youtube= build('youtube','v3',developerKey=api_key)

In [3]:
def get_channel(youtube,channel_ids):
    
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids)
                )
    response = request.execute() 
    

    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)




In [4]:
channel_stats=get_channel(youtube,channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,MrBeast,257000000,48402247576,793,UUX6OQ3DkcsbYNE6H8uQQuVA
1,Rohit & Kanupriya,2280000,2166832637,326,UUtAXJ4DNpshfVjLIMq9pIRw
2,Something_beyond,18,8543,46,UUkX0NXCP9kCLtk30kjTXIWg
3,Darshil Parmar,129000,5938594,149,UUChmJrVa8kDg05JfCmxpLRw
4,KEEMOKAZI,9740000,10609455457,1332,UUMOLiZpKXp-w5CNfqJObQUw
5,KALEO,1940000,1553977588,68,UUXqJrBG563pZnaqLoxHjUDg
6,coder2j,9920,921193,48,UUHoLIMtg_OigNlJmjHX9J8Q


In [5]:
playlistid=channel_stats['playlistId']
playlistid

0    UUX6OQ3DkcsbYNE6H8uQQuVA
1    UUtAXJ4DNpshfVjLIMq9pIRw
2    UUkX0NXCP9kCLtk30kjTXIWg
3    UUChmJrVa8kDg05JfCmxpLRw
4    UUMOLiZpKXp-w5CNfqJObQUw
5    UUXqJrBG563pZnaqLoxHjUDg
6    UUHoLIMtg_OigNlJmjHX9J8Q
Name: playlistId, dtype: object

In [6]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids


In [7]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    
Video_ids

['F6PqxbvOCUI',
 'oA4LSZvX4iE',
 'ZKxnjszkZto',
 '8_gdcaX9Xqk',
 'Pv0iVoSZzN8',
 'CWbV3NItSdY',
 'l-nMKJ5J3Uc',
 'XE1Qyss8GIY',
 '6GzHPS0rEgc',
 'imhY0pe-Sd8',
 'erLbbextvlY',
 'OnTTThIzuNU',
 'mKdjycj-7eE',
 'tWYsfOSY9vY',
 'sH48lKbayNU',
 'AFXoSFNMwIA',
 'AjwAdtlRKf8',
 'KOEfDvr4DcQ',
 'krsBRQbOPQ4',
 '7ESeQBeikKs',
 'K_CbgLpvH9E',
 'lOKASgtr6kU',
 '9RhWXPcKBI8',
 'ZVt9ZJfWV1c',
 'rWBOITBjitE',
 'tnTPaLOaHz8',
 'Wdjh81uH6FU',
 '7dYTw-jAYkY',
 'AjZDwxt7S8w',
 'aZnqqXaT02g',
 'mwKJfNYwvm8',
 'l9_8_pDTmis',
 'n7x4Jj9pdH8',
 'QjvpjXdgugA',
 'jGAJCAuV3pQ',
 'c0v8OPItCOg',
 '3ryID_SwU5E',
 'IemzxkkzI5s',
 '3OFj6l2tQ9s',
 'bvlFjrcpD6s',
 'AZlBZuS4hr0',
 'Gx6VeIIbWwM',
 'CAOY8EKH1qU',
 'IQxea9UB1nQ',
 'KrLj6nc516A',
 'zgLhCT7aFCA',
 'L5kIXRdwGAI',
 '19NmWqv4Alk',
 'vBpQ1SlfVtU',
 'NnNZLCl78Ao',
 'NTZhwAldqwg',
 'J_z-W4UVHkw',
 'mKIhHNznt4s',
 'jObOjhUkf50',
 'O3YVBJ2n5xg',
 'yhB3BgJyGl8',
 'CP82DAKzL1U',
 'fuhE6PYnRMc',
 'stUofW3whpE',
 '48h57PspBec',
 'ooAwCOP67GQ',
 'FM7Z-Xq8Drc',
 'WTOm65

In [8]:
def get_video_details(youtube, video_ids):    
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [9]:
videoDetails=get_video_details(youtube, Video_ids)
videoDetails

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,F6PqxbvOCUI,MrBeast,"Protect The Yacht, Keep It!",This video did not go how I expected it to\nCr...,None,2024-05-11T16:00:00Z,1296896,196649,None,16358,PT15M8S,hd,true
1,oA4LSZvX4iE,MrBeast,Ages 1-100 Try My Chocolate,,None,2024-05-09T16:00:00Z,19864376,1447579,None,8955,PT39S,hd,false
2,ZKxnjszkZto,MrBeast,"Spot The Hidden People For $10,000",,None,2024-05-08T16:00:01Z,28928710,1538699,None,5115,PT35S,hd,false
3,8_gdcaX9Xqk,MrBeast,"Would You Split Or Steal $250,000?",,None,2024-05-03T16:00:00Z,38168140,3405014,None,18733,PT1M,hd,false
4,Pv0iVoSZzN8,MrBeast,10 Minutes To Escape Or This Room Explodes!,I didn’t expect that to happen…\nThere’s no ju...,None,2024-04-27T16:00:00Z,82462392,3440891,None,86971,PT10M,hd,true
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,CLkzXrjrFKg,coder2j,Airflow DAG: create dag with bash operator!,Airflow DAG: create dag with bash operator!\n\...,"[python, etl, datapipeline, dataengineer, airf...",2021-08-09T04:00:05Z,24216,369,None,71,PT12M50S,hd,false
2759,UFsCvWjQT4w,coder2j,Airflow Task Lifecycle and Architecture: Airfl...,Airflow Task Lifecycle and Architecture: Airfl...,"[python, etl, datapipeline, dataengineer, airf...",2021-06-07T10:10:00Z,16189,284,None,25,PT7M13S,hd,false
2760,mtJHMdoi_Gg,coder2j,Airflow Core Concepts in 5 mins: Airflow Tutor...,Airflow Core Concepts in 5 mins: Airflow Tutor...,"[python, etl, datapipeline, dataengineer, airf...",2021-03-10T09:06:41Z,20450,380,None,21,PT4M50S,hd,false
2761,J6azvFhndLg,coder2j,Airflow Docker: run Airflow 2.0 in docker cont...,Airflow Docker: run Airflow 2.0 in docker cont...,"[airflow, airflow tutorial, apache airflow, ap...",2021-03-02T21:29:35Z,34377,382,None,82,PT8M25S,hd,false


In [10]:
videoDetails.to_csv('Data\\apiData.csv', index=False)